This notebook compares analytical and numerical evaluations of alchemical derivatives of a CO molecule up to the third order in KSDFT.  
The results are summerized in the last section.

In [1]:
from pyscf import gto,scf
import numpy as np
import pyscf

pyscf.__version__

'2.2.1'

In [2]:
from FcMole import FcM_like

In [3]:
target_mol = "C 0 0 0; O 0 0 2.1"
dft_functional = "pbe0"  # "lda,vwn"
# basis_set = "def2-TZVP"
import basis_set_exchange as bse
basis_set = bse.get_basis('pcX-2',fmt="nwchem")

### Analytical evaluation

In [4]:
%load_ext autoreload
from AP_class import APDFT_perturbator as AP

In [5]:
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
mf_nn=scf.RKS(mol_NN)
mf_nn.xc = dft_functional
mf_nn.scf()
# mf_nn.conv_tol = 1e-12
# mf_nn.grids.level = 6
# mf_nn.nuc_grad_method().kernel()
ap_nn=AP(mf_nn,sites=[0,1],flag_response_property=True)

converged SCF energy = -113.231744670484


In [6]:
analytical_1st_alchemical_derivative = ap_nn.build_gradient()
analytical_2nd_alchemical_derivative = ap_nn.build_hessian()
analytical_3rd_alchemical_derivative = ap_nn.build_cubic()

Inefficient alchemical force in KS-DFT which calculates the response matrix for nuclear coordinates

In [7]:
inefficient_analytical_alchemical_force = np.zeros((2, 2, 3))
inefficient_analytical_alchemical_force[0] = ap_nn.build_inefficient_alchemical_force(0)
inefficient_analytical_alchemical_force[1] = ap_nn.build_inefficient_alchemical_force(1)

Efficient alchemical force in KS-DFT which calculates the response matrix for nuclear coordinates

In [8]:
efficient_analytical_alchemical_force = np.zeros((2, 2, 3))
efficient_analytical_alchemical_force[0] = ap_nn.build_alchemical_force(0)
efficient_analytical_alchemical_force[1] = ap_nn.build_alchemical_force(1)

Electric dipole derivatives $\partial ^2 E/ \partial Z_I \partial \mathbf{F}$

In [9]:
elec_electric_dipole_gradient_z = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='Z')
elec_electric_dipole_gradient_f = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='F')

Electric polarizability derivatives $\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F}$

In [10]:
elec_electric_pol_gradient = ap_nn.build_electric_polarizability_gradient()

### Numerical evaluation

In [11]:
%autoreload 2
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
fc_param = 0.001
fcs1 = [fc_param, 0.0]
fcs2 = [-fc_param, 0.0]
fcs3 = [0.0, fc_param]
fcs4 = [0.0, -fc_param]

##### Efficient numerical alchemical force

In [12]:
fmol1=FcM_like(mol_NN,fcs=fcs1)
fmol2=FcM_like(mol_NN,fcs=fcs2)
fmol3=FcM_like(mol_NN,fcs=fcs3)
fmol4=FcM_like(mol_NN,fcs=fcs4)
mf=scf.RKS(mol_NN)
mf1=scf.RKS(fmol1)
mf2=scf.RKS(fmol2)
mf3=scf.RKS(fmol3)
mf4=scf.RKS(fmol4)
mf.xc = dft_functional
mf1.xc = dft_functional
mf2.xc = dft_functional
mf3.xc = dft_functional
mf4.xc = dft_functional
# mf.conv_tol = 1e-12
# mf1.conv_tol = 1e-12
# mf2.conv_tol = 1e-12
# mf3.conv_tol = 1e-12
# mf4.conv_tol = 1e-12
# mf.grids.level = 6
# mf1.grids.level = 6
# mf2.grids.level = 6
# mf3.grids.level = 6
# mf4.grids.level = 6
# Without dm0=mf.init_guess_by_1e(), some SCFs do not converge.
e=mf.scf(dm0=mf.init_guess_by_1e())
e1=mf1.scf(dm0=mf1.init_guess_by_1e())
e2=mf2.scf(dm0=mf2.init_guess_by_1e())
e3=mf3.scf(dm0=mf3.init_guess_by_1e())
e4=mf4.scf(dm0=mf4.init_guess_by_1e())

converged SCF energy = -113.231744610727


/home/takafumi/program/pyscf/pyscf/gto/mole.py:1213: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -113.246440595758
converged SCF energy = -113.217051598173
converged SCF energy = -113.254005307678
converged SCF energy = -113.209487562025


In [13]:
nuc_grad = mf.nuc_grad_method().kernel()
nuc_grad1 = mf1.nuc_grad_method().kernel()
nuc_grad2 = mf2.nuc_grad_method().kernel()
nuc_grad3 = mf3.nuc_grad_method().kernel()
nuc_grad4 = mf4.nuc_grad_method().kernel()

--------------- RKS gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000     0.0290752699
1 O    -0.0000000000     0.0000000000    -0.0290813083
----------------------------------------------
--------------- RKS gradients ---------------
         x                y                z
0 C    -0.0000000000    -0.0000000000     0.0288517909
1 O     0.0000000000     0.0000000000    -0.0288578264
----------------------------------------------
--------------- RKS gradients ---------------
         x                y                z
0 C    -0.0000000000    -0.0000000000     0.0292987915
1 O     0.0000000000     0.0000000000    -0.0293048327
----------------------------------------------
--------------- RKS gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000     0.0290286955
1 O    -0.0000000000    -0.0000000000    -0.0290347423
----------------------------------------------
--------

In [14]:
efficient_numerical_alchemical_force = np.zeros((2, 2, 3))
efficient_numerical_alchemical_force[0] = (nuc_grad1 - nuc_grad2) / (2 * fc_param)
efficient_numerical_alchemical_force[1] = (nuc_grad3 - nuc_grad4) / (2 * fc_param)

#### Numerical alchemical derivatives

In [15]:
numerical_1st_alchemical_derivative = np.zeros((2))
numerical_1st_alchemical_derivative[0] = (e1 - e2) / (2 * fc_param)
numerical_1st_alchemical_derivative[1] = (e3 - e4) / (2 * fc_param)

#### Numerical alchemical hardness

In [16]:
ap_nn=AP(mf,sites=[0,1])
ap_nn1=AP(mf1,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
ap_nn2=AP(mf2,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
ap_nn3=AP(mf3,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
ap_nn4=AP(mf4,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
an = ap_nn.build_gradient()
an1 = ap_nn1.build_gradient()
an2 = ap_nn2.build_gradient()
an3 = ap_nn3.build_gradient()
an4 = ap_nn4.build_gradient()
ann = ap_nn.build_hessian()
ann1 = ap_nn1.build_hessian()
ann2 = ap_nn2.build_hessian()
ann3 = ap_nn3.build_hessian()
ann4 = ap_nn4.build_hessian()
dipole1 = ap_nn1.ref_elec_electric_dipole_moment * -1
dipole2 = ap_nn2.ref_elec_electric_dipole_moment * -1
dipole3 = ap_nn3.ref_elec_electric_dipole_moment * -1
dipole4 = ap_nn4.ref_elec_electric_dipole_moment * -1
pol1 = ap_nn1.ref_elec_electric_polarizability * -1
pol2 = ap_nn2.ref_elec_electric_polarizability * -1
pol3 = ap_nn3.ref_elec_electric_polarizability * -1
pol4 = ap_nn4.ref_elec_electric_polarizability * -1

In [17]:
numerical_2nd_alchemical_derivative = np.zeros((2, 2))
numerical_2nd_alchemical_derivative[0] = (an1 - an2) / (2 * fc_param)
numerical_2nd_alchemical_derivative[1] = (an3 - an4) / (2 * fc_param)

#### Numerical alchemical hardness derivatives

In [18]:
numerical_3rd_alchemical_derivative = np.zeros((2, 2, 2))
numerical_3rd_alchemical_derivative[0, :, :] = (ann1 - ann2) / (2 * fc_param)
numerical_3rd_alchemical_derivative[1, :, :] = (ann3 - ann4) / (2 * fc_param)

#### Inefficient numerical alchemical force

In [19]:
mol_NN1=gto.M(atom="C 0 0 0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
mol_NN2=gto.M(atom="C 0 0 -0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
mol_NN3=gto.M(atom="C 0 0 0; O 0 0 2.101",unit="Bohr",basis=basis_set)
mol_NN4=gto.M(atom="C 0 0 0; O 0 0 2.099",unit="Bohr",basis=basis_set)
mf_nn1=scf.RKS(mol_NN1)
mf_nn2=scf.RKS(mol_NN2)
mf_nn3=scf.RKS(mol_NN3)
mf_nn4=scf.RKS(mol_NN4)
mf_nn1.xc = dft_functional
mf_nn2.xc = dft_functional
mf_nn3.xc = dft_functional
mf_nn4.xc = dft_functional
mf_nn1.scf()
mf_nn2.scf()
mf_nn3.scf()
mf_nn4.scf()
ap_nn1=AP(mf_nn1,sites=[0,1])
ap_nn2=AP(mf_nn2,sites=[0,1])
ap_nn3=AP(mf_nn3,sites=[0,1])
ap_nn4=AP(mf_nn4,sites=[0,1])
ad1 = ap_nn1.build_gradient()
ad2 = ap_nn2.build_gradient()
ad3 = ap_nn3.build_gradient()
ad4 = ap_nn4.build_gradient()

converged SCF energy = -113.231714896515
converged SCF energy = -113.231773036051
converged SCF energy = -113.231773036051
converged SCF energy = -113.231714896515


In [20]:
inefficient_numerical_alchemical_force = np.zeros((2, 2))  # only z axis components
inefficient_numerical_alchemical_force[:, 0] = (ad1 - ad2) / (2 * fc_param)
inefficient_numerical_alchemical_force[:, 1] = (ad3 - ad4) / (2 * fc_param)

In [21]:
numer_elec_electric_dipole_gradient = np.zeros((2, 3))
numer_elec_electric_dipole_gradient[0] = (dipole1 - dipole2) / (2 * fc_param)
numer_elec_electric_dipole_gradient[1] = (dipole3 - dipole4) / (2 * fc_param)

In [22]:
numer_elec_electric_pol_gradient = np.zeros((2, 3, 3))
numer_elec_electric_pol_gradient[0] = (pol1 - pol2) / (2 * fc_param)
numer_elec_electric_pol_gradient[1] = (pol3 - pol4) / (2 * fc_param)

## Comparison of analytical and numerical evaluations

$ \partial E/\partial Z_i $

In [23]:
print(analytical_1st_alchemical_derivative)

[-14.69449855 -22.25887343]


$ (\partial E/\partial Z_i)_\mathrm{analytical} $ - $ (\partial E/\partial Z_i)_\mathrm{numerical} $

In [24]:
print(analytical_1st_alchemical_derivative - numerical_1st_alchemical_derivative)

[ 2.43462971e-07 -6.00392791e-07]


$\partial^2E/\partial Z_i\partial Z_j$

In [25]:
print(analytical_2nd_alchemical_derivative)

[[-2.97247707  0.45684296]
 [ 0.45684296 -3.64824864]]


$(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{numerical}$

In [26]:
print(analytical_2nd_alchemical_derivative - numerical_2nd_alchemical_derivative)

[[ 6.15265903e-06 -1.95087793e-05]
 [ 2.99294388e-06 -7.24907511e-06]]


$\partial^3E/\partial Z_i\partial Z_j\partial Z_k$

In [27]:
print(analytical_3rd_alchemical_derivative)

[[[-0.20640009  0.11823329]
  [ 0.11823329  0.08695088]]

 [[ 0.11823329  0.08695088]
  [ 0.08695088 -0.21972794]]]


$(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{analytical}$ - $(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{numerical}$

In [28]:
print(analytical_3rd_alchemical_derivative - numerical_3rd_alchemical_derivative)

[[[ 2.01063655e-06  2.35563893e-07]
  [ 2.35563893e-07 -4.62388347e-07]]

 [[ 8.72882508e-07  2.44112372e-07]
  [ 2.44112372e-07 -3.65192599e-07]]]


$\partial^2E/\partial Z_i\partial R_j$

In [29]:
print(efficient_analytical_alchemical_force)

[[[-1.50760059e-14 -9.51351765e-14 -2.23495259e-01]
  [ 1.51582599e-14  9.51612247e-14  2.23498077e-01]]

 [[ 1.41036059e-13 -1.03564289e-13 -4.66604450e-02]
  [-1.40682135e-13  1.03472292e-13  4.66520177e-02]]]


$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [30]:
print(efficient_analytical_alchemical_force - efficient_numerical_alchemical_force)

[[[-1.38854382e-12  9.94914802e-13  5.04179004e-06]
  [ 1.22058931e-12 -2.91266232e-13 -5.08202108e-06]]

 [[ 5.78868073e-13 -5.20717026e-12  1.93308889e-06]
  [-2.73201750e-13  5.42365809e-12 -1.94148468e-06]]]


$\partial^2E/\partial Z_i\partial R_j$

In [31]:
print(inefficient_analytical_alchemical_force)

[[[-1.50734325e-14 -9.51287797e-14 -2.23495259e-01]
  [ 1.51510156e-14  9.51609635e-14  2.23498077e-01]]

 [[ 1.41053514e-13 -1.03568484e-13 -4.66604441e-02]
  [-1.40664395e-13  1.03458470e-13  4.66520168e-02]]]


$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [32]:
print(inefficient_analytical_alchemical_force[:, :, 2] - inefficient_numerical_alchemical_force)

[[ 5.64846060e-06 -2.83030876e-06]
 [-1.07578930e-05  2.32983493e-06]]


$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$

In [33]:
print(elec_electric_dipole_gradient_f)

[[ 2.28380933e-14 -1.02141902e-13 -6.58025487e-01]
 [ 3.07790443e-13 -5.12289082e-14  1.19452177e+00]]


In [34]:
print(elec_electric_dipole_gradient_z)

[[ 2.27719361e-14 -1.01910326e-13 -6.58025487e-01]
 [ 3.08021486e-13 -5.12984307e-14  1.19452177e+00]]


$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$ - $(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{numerical}$

In [35]:
print(elec_electric_dipole_gradient_f - numer_elec_electric_dipole_gradient)

[[-3.09935843e-11 -5.65701264e-11  3.04231699e-05]
 [ 6.47596065e-11  1.07134585e-11  1.44385444e-05]]


In [36]:
print(elec_electric_dipole_gradient_z - numer_elec_electric_dipole_gradient)

[[-3.09936505e-11 -5.65698948e-11  3.04233211e-05]
 [ 6.47598376e-11  1.07133890e-11  1.44372652e-05]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$

In [37]:
print(elec_electric_pol_gradient)

[[[ 6.91330618e+00 -1.74440217e-13 -5.45702537e-14]
  [-1.74440217e-13  6.91330618e+00  2.83125696e-13]
  [-5.45702537e-14  2.83125696e-13  6.35332248e+00]]

 [[ 3.16207069e+00 -3.14367857e-13  1.32270441e-12]
  [-3.14367857e-13  3.16207069e+00 -7.34142840e-13]
  [ 1.32270441e-12 -7.34142840e-13  7.36584289e+00]]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$ - $(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{numerical}$

In [38]:
print(elec_electric_pol_gradient - numer_elec_electric_pol_gradient)

[[[-1.05312001e-05  3.56623484e-10  9.06634668e-10]
  [ 3.56623484e-10 -1.05281862e-05 -4.98390635e-10]
  [ 9.06634668e-10 -4.98390635e-10  8.13403593e-05]]

 [[ 4.58669462e-07  2.78714192e-11  4.83359315e-11]
  [ 2.78714192e-11  4.58666097e-07  3.97630918e-11]
  [ 4.83359315e-11  3.97630918e-11  3.67467674e-05]]]
